In [1]:
%run GpuOptions.ipynb
%run BuildTrainingData.ipynb
%run TrendTools.ipynb

ModuleNotFoundError: No module named 'nbformat'

In [7]:
import os
import tensorflow as tf
from tensorflow.keras.layers import *
import common.model as m

classes = training_data_generator.class_indices
data, _ = training_data_generator.next()
shape = data[0].shape
inputs = tf.keras.Input(shape=shape)

outputs = m.chained(
    # Entry Flow
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    m.expand(
        flow=lambda previous_activation, size: tf.keras.layers.add([
            m.chained(
                m.duplicate(
                    layers=lambda: [
                        tf.keras.layers.Activation('relu'),
                        tf.keras.layers.SeparableConv2D(filters=size, kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization(),
                    ],
                    count=2
                ),
                tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
            )(previous_activation),
            tf.keras.layers.Conv2D(filters=size, kernel_size=1, strides=2, padding='same')(previous_activation),
        ]),
        values=[128, 256, 728],
    ),

    # # Middle Flow
    m.expand(
        flow=lambda previous_activation, _: tf.keras.layers.add([
            m.duplicate(
                layers=lambda: [
                    tf.keras.layers.Activation('relu'),
                    tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                ],
                count=3,
            )(previous_activation),
            previous_activation,
        ]),
        values=[0] * 8
    ),

    # # Exit Flow
    lambda previous_activation: tf.keras.layers.add([
        m.chained(
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
        )(previous_activation),
        tf.keras.layers.Conv2D(filters=1024, kernel_size=1, strides=2, padding='same')(previous_activation),
    ]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(classes), activation='linear'),
    tf.keras.layers.Dropout(0.2),
)(inputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.build(input_shape=(None, *shape))
model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.08),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.03),
    metrics=['accuracy'],
)
if os.path.exists('isthemountainout.best.h5'):
    model.load_weights('isthemountainout.best.h5')
elif os.path.exists('isthemountainout.h5'):
    model.load_weights('isthemountainout.h5')
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 112, 112, 32) 896         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_108 (BatchN (None, 112, 112, 32) 128         conv2d_18[0][0]                  
__________________________________________________________________________________________________
activation_108 (Activation)     (None, 112, 112, 32) 0           batch_normalization_108[0][0]    
_______________________________________________________________________________________

In [ ]:
from datetime import datetime

model.fit(
    training_data_generator,
    epochs=700,
    verbose=True,
    steps_per_epoch=training_data_generator.samples // 
        training_data_generator.batch_size,
    validation_data=validation_data_generator,
    validation_steps=validation_data_generator.samples // 
        validation_data_generator.batch_size,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            'isthemountainout.best.h5',
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            verbose=True),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=20,
            restore_best_weights=True,
            verbose=True),
        tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join('tensor-logs', 'fit', datetime.now().strftime('%Y%m%d%H%M%S')),
            update_freq=50,
            write_images=True,
            write_graph=True,
            embeddings_freq=10),
    ])

Epoch 1/700
1673/1673 [==============================] - ETA: 0s - loss: 0.5246 - accuracy: 0.8220
Epoch 00001: val_loss improved from inf to 0.57514, saving model to isthemountainout.best.h5
1673/1673 [==============================] - 1925s 1s/step - loss: 0.5246 - accuracy: 0.8220 - val_loss: 0.5751 - val_accuracy: 0.8319
Epoch 2/700
1673/1673 [==============================] - ETA: 0s - loss: 0.4733 - accuracy: 0.8422
Epoch 00002: val_loss improved from 0.57514 to 0.55877, saving model to isthemountainout.best.h5
1673/1673 [==============================] - 1902s 1s/step - loss: 0.4733 - accuracy: 0.8422 - val_loss: 0.5588 - val_accuracy: 0.8164
Epoch 3/700
1673/1673 [==============================] - ETA: 0s - loss: 0.4215 - accuracy: 0.8631
Epoch 00003: val_loss improved from 0.55877 to 0.49266, saving model to isthemountainout.best.h5
1673/1673 [==============================] - 1914s 1s/step - loss: 0.4215 - accuracy: 0.8631 - val_loss: 0.4927 - val_accuracy: 0.8577
Epoch 4/700